In [198]:
import cr_interface as cri
import numpy as np
import pandas as pd
import sklearn as skl

In [206]:
#results = cri.load_results(results_dir="over")
results = cri.load_results()
metadata = cri.load_metadata()

JSONDecodeError: Expecting ',' delimiter: line 1 column 102660 (char 102659)

In [200]:
results.sort(key=lambda d: d['test_accuracy'])
best_result = results[-1]
best_result['test_accuracy']

'0.77208483'

In [201]:
selected_result = best_result

In [202]:
def get_result_df(result, training=False):
    if training:
        cr_codes = map(cri.extract_cr_code, result['training_images'])
    else:
        cr_codes = map(cri.extract_cr_code, result['predictions'].keys())
        
    cr_codes = list(set(cr_codes))
    
    for i, cr_code in enumerate(cr_codes):
        cr_codes[i] = 'D00' + cr_code[3:]

    extract_dataset = lambda cr_code: cri.parse_cr_code(cr_code)[0]
    extract_pid = lambda cr_code: cri.parse_cr_code(cr_code)[1]
    extract_phase = lambda cr_code: cri.parse_cr_code(cr_code)[2]
    extract_slice = lambda cr_code: cri.parse_cr_code(cr_code)[3]

    datasets = list(map(extract_dataset, cr_codes))
    pids = list(map(extract_pid, cr_codes))
    phases = list(map(extract_phase, cr_codes))
    slices = list(map(extract_slice, cr_codes))

    extract_label = lambda cr_code: metadata[cr_code]['label']
    labels = list(map(extract_label, cr_codes))
    
    # make tri labels
    for i, label in enumerate(labels):
        if label in ['ap', 'bs', 'md']:
            labels[i] = 'in'
        
    d = dict(
            cr_code=cr_codes,
            dataset=datasets,
            pid=pids,
            phase=phases,
            slice=slices,
            label=labels,
    )
    
    if not training:
        extract_prediction = lambda cr_code: result['predictions']['testing_{}'.format(cr_code)]['prediction']
        predictions = list(map(extract_prediction, cr_codes))
        d['predictions'] = predictions
        

    return pd.DataFrame(d)

In [203]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['predictions'])

predictions,in,oap,obs
label,,,
in,152,7,7
oap,11,38,0
obs,10,2,19


In [204]:
df = get_result_df(selected_result, training=False)
df = df.drop_duplicates()
pd.crosstab(df['label'], df['predictions'])

precision = {} # 정밀도
recall = {} # 재현율

lp = df[['label', 'predictions']]
for label in set(df['label']):
    true_positives = lp[lambda e: (e.label == label) & (e.predictions == label)]
    truths = lp[lambda e: e.label == label]
    positives = lp[lambda e: e.predictions == label]
    
    precision[label] = len(true_positives) / len(positives)
    recall[label] = len(true_positives) / len(truths)

summary_df = pd.DataFrame(dict(
    precision=precision,
    recall=recall
))
summary_df

,precision,recall
in,0.878613,0.915663
oap,0.808511,0.775510
obs,0.730769,0.612903
